In [1]:
!pip install flask tensorflow nltk pyngrok
!python -m nltk.downloader punkt wordnet


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 112, in _get_module_details
  File "C:\Users\Shanika\AppData\Roaming\Python\Python312\site-packages\nltk\__init__.py", line 146, in <module>
    from nltk.chunk import *
  File "C:\Users\Shanika\AppData\Roaming\Python\Python312\site-packages\nltk\chunk\__init__.py", line 155, in <module>
    from nltk.chunk.api 

In [3]:
from flask import Flask, render_template, request , jsonify
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
import random
import json
import pickle
from tensorflow.keras.models import load_model
from pyngrok import ngrok

Initialize Flask App

In [5]:
app = Flask(__name__, template_folder='../templates',static_folder='../static')

load chatbot model and data

In [13]:
print("loading chatbot resources...")
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json', encoding='utf-8').read())
words = pickle.load(open('../words.pkl', 'rb'))
classes = pickle.load(open('../classes.pkl', 'rb'))
model = load_model('../chatbot_model.h5')
print("Chatbot resources loaded succesfully!")

loading chatbot resources...


Chatbot resources loaded succesfully!


Chatbot Functions

In [15]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    return [lemmatizer.lemmatize(word.lower()) for word in sentence_words]

def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)
    
def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list
    
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    for i in intents_json['intents']:
        if i['tag'] == tag:
            return random.choice(i['responses'])
    return "I'm not sure I understand. Could you rephrase that?"

Flask Routes

In [17]:
@app.route('/')
def home():
    return render_template('index.html')

@app.route('/get_response', methods=['POST'])
def get_bot_response():
    user_message = request.json['message']
    ints = predict_class(user_message)
    response = get_response(ints, intents)
    return jsonify({'response': response})

Run Flask App with Ngrok

In [ ]:
# Cell 7: Run Flask App with Ngrok (Updated)
if __name__ == '__main__':
    try:
        # Configure ngrok with alternative region
        ngrok.set_auth_token("2xZgWkaSN7aIb1DxtGZ0p8Hpk1d_24Hv9XVQfafrGduqVoPgK")  
        public_url = ngrok.connect(5000, region='ap') 
        print(f"\n * Public URL: {public_url}")
        print(" * This URL can be used to access your app from anywhere")
        print(" * Use this for WhatsApp webhook if needed\n")
    except Exception as e:
        print(f"\n * Ngrok connection failed: {str(e)}")
        print(" * Running locally only (accessible via http://localhost:5000)\n")
    
    # Run Flask app
    app.run(host='0.0.0.0', port=5000)


 * Ngrok connection failed: ngrok client exception, API returned 400: {"error_code":102,"status_code":400,"msg":"invalid tunnel configuration","details":{"err":"yaml: unmarshal errors:\n  line 1: field region not found in type config.HTTPv2Tunnel"}}

 * Running locally only (accessible via http://localhost:5000)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.126.226:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [26/May/2025 09:50:46] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/May/2025 09:50:46] "GET /static/style.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [26/May/2025 09:50:46] "GET /static/script.js HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [26/May/2025 09:50:46] "GET /favicon.ico HTTP/1.1" 404 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 277ms/step


INFO:werkzeug:127.0.0.1 - - [26/May/2025 09:51:01] "POST /get_response HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step


...

INFO:werkzeug:127.0.0.1 - - [26/May/2025 09:54:41] "POST /get_response HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/May/2025 10:02:53] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/May/2025 10:02:53] "GET /static/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/May/2025 10:02:53] "GET /static/script.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/May/2025 10:03:27] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/May/2025 10:03:27] "GET /static/style.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [26/May/2025 10:03:27] "GET /static/script.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/May/2025 10:03:51] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/May/2025 10:03:51] "GET /static/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/May/2025 10:03:51] "GET /static/script.js HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [26/May/2025 10:04:00] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/May/2025 10:04:00] "GET /static/style.css HTT

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 818ms/step


INFO:werkzeug:127.0.0.1 - - [26/May/2025 10:20:14] "POST /get_response HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step


INFO:werkzeug:127.0.0.1 - - [26/May/2025 10:20:22] "POST /get_response HTTP/1.1" 200 -
